# Exercise

- Use the previous knowledge to extract information from the European Environment Agency (EEA). 
- Adopt the monitoring co2 emissions [dataset](http://www.eea.europa.eu/data-and-maps/data/co2-cars-emission-11) (**Monitoring of CO2 emissions from passenger cars - Data 2015 - Final data**) 

In [1]:
#%%timeit
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from sklearn import linear_model
from pprint import pprint
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.charts import Bar
from bokeh.models.widgets import Tabs, Panel

In [2]:
pd_cars = pd.read_csv("versao3.csv", low_memory = False, error_bad_lines = False, warn_bad_lines = False, encoding = "utf-8")
fuelfilter = ['E85', 'DIESEL-ELECTRIC', 'PETROL-ELECTRIC', 'ELECTRIC', 'NG-BIOMETHANE', 'LPG', 'PETROL', 'DIESEL']

In [3]:
def Filter():
    df_filter = []
    for index, row in pd_cars.iterrows():
        try:
            if (str(row["ep (KW)"]).isnumeric()) and (str(row["e (g/km)"]).isnumeric()) and fuelfilter.index(str(row["Ft"])):
                df_filter.append(row)
        except:
            continue

    df_filter = pd.DataFrame(df_filter)        
    len(df_filter)
    df_filter.to_csv("versao3.csv",  encoding = "utf-8")

#Filter()

# Return array 2D
def CountValues(index, valMin):
    count = 0
    listReturn = []
    for i, row in pd_cars[index].value_counts().sort_values().iteritems():
        if(row > valMin):
            count += row
            listReturn.append([i, row, 0, 0])
    np_listReturn = np.array(listReturn)   
    print (count)
    return np_listReturn

np_mh é um array 2d que fica o nome das fabricantes e a quantidade de carros que elas fizeram. 
Esse array tem 8 fabricantes e 216.853 carros. Todas as fabricantes tem mais que 10.000 carros produzidos.

In [4]:
np_mh = CountValues("Mh", 10000)
print(len(np_mh[:,0]))

data = {
    'Fabricantes': np_mh[:,0],
    'Quantidade de carros': np_mh[:,1]
}

# x-axis labels pulled from the fuelName column, stacking labels from sample column
bar = Bar(data, values='Quantidade de carros', label='Fabricantes', agg="mean",
          title="Fabricantes de carros", legend=False, plot_width=900)

p = figure(width=400, height=400)
p.vbar(x=np_mh[:,1], width=0.5, bottom=0, top=np_mh[:,1])

output_notebook()
show(bar)

216853
8


Loading BokehJS ...

In [5]:
for row in np_mh:    
    x = pd_cars[pd_cars["Mh"] == row[0]]["ep (KW)"]
    y = pd_cars[pd_cars["Mh"] == row[0]]["e (g/km)"]
    #Calculates a Pearson correlation coefficient and the p-value for testing non-correlation.
    row[2] = pearsonr(x, y)[0]
    row[3] = (pearsonr(x, y)[0]**2)*100
    
#print(np_mh[np_mh[:, 2].argsort()])
first = np_mh[np_mh[:, 2].argsort()][:2]
last = np_mh[np_mh[:, 2].argsort()][-2:]
np_mh_job = np.concatenate((first, last), axis=0)
#print(np_mh_job[0][0])

In [6]:
listTabs = []
for row in np_mh_job:
    x_values = pd_cars[pd_cars["Mh"] == row[0]][["ep (KW)"]]
    y_values = pd_cars[pd_cars["Mh"] == row[0]][["e (g/km)"]]
    #train model on data
    bmi_life_model = linear_model.LinearRegression()
    bmi_life_model.fit(x_values, y_values)
    predict = bmi_life_model.predict(x_values).ravel()

    x = pd_cars[pd_cars["Mh"] == row[0]]["ep (KW)"]
    y = pd_cars[pd_cars["Mh"] == row[0]]["e (g/km)"]

    p = figure(plot_width=800, plot_height=600, title=row[0], x_axis_label='ep (KW)', y_axis_label='e (g/km)')

    p.line(x, predict, line_width=2)
    p.circle(x, y, fill_color="white", size=5)
    label = str(row[0]) + "  |  R² = " + str(row[3])[0:5]
    listTabs.append(Panel(child=p, title=label))

layout = Tabs(tabs=listTabs)

# Call the output_notebook() 
output_notebook()
show(layout)

Loading BokehJS ...

In [13]:
pot = []
for row in np_mh_job:
    listAux = []
    for i, rowPot in pd_cars[pd_cars["Mh"] == row[0]]["ep (KW)"].value_counts()[0:5].iteritems():
        #print (i, rowPot)
        listAux.append([i, rowPot])
    pot.append(listAux)

pot = np.array(pot)
np.sort(pot[0][:,0]) 

pd_cars.groupby(['Ft']).sum()

,Unnamed: 0,id,r,e (g/km),m (kg),w (mm),at1 (mm),at2 (mm),ec (cm3),ep (KW),z (Wh/km),Unnamed: 27,Unnamed: 28,Unnamed: 29
Ft,,,,,,,,,,,,,,
DIESEL,35813674818,39683941686,4957414,22358059,283827838.0,470370728.0,266730152.0,251598978.0,337474387.0,19617101,NaN,216.0,NaN,NaN
DIESEL-ELECTRIC,113930244,73258794,9614,33812,540430.0,909292.0,521309.0,495274.0,635601.0,36036,6404.0,NaN,NaN,NaN
ELECTRIC,346500778,254777898,38250,0,2400867.0,3529932.0,1992767.0,1848297.0,NaN,149678,197479.0,NaN,NaN,NaN
LPG,453724026,568431143,130836,237589,2446984.0,4551479.0,2617894.0,2143480.0,2621783.0,144739,NaN,NaN,NaN,NaN
NG-BIOMETHANE,314739219,321652680,75658,155009,2012790.0,3355500.0,2023833.0,1687858.0,1986396.0,114714,NaN,NaN,NaN,NaN
PETROL,27397609484,30505128171,4403177,18506910,182279076.0,327344504.0,190472290.0,183232968.0,227474065.0,16108447,548.0,NaN,NaN,NaN
PETROL-ELECTRIC,224400824,287134886,49515,83605,2269876.0,3274495.0,1871371.0,1777801.0,2497212.0,194757,107007.0,NaN,NaN,NaN


In [8]:
from bokeh.charts import Line

data = dict(VOLKSWAGEN = np.sort(pot[0][:,0]),
            FORD = np.sort(pot[1][:,0]),
            BMW = np.sort(pot[2][:,0]),
            AUDI = np.sort(pot[3][:,0])
            )
df = pd.DataFrame(data)


line2 = Line(df, y=['VOLKSWAGEN', 'FORD', 'BMW', 'AUDI'],
             dash=['VOLKSWAGEN', 'FORD', 'BMW', 'AUDI'],
             title="Interpreters", ylabel='Potencia', legend=True)

show(line2)

In [9]:
import pandas as pd

from bokeh.layouts import gridplot
from bokeh.charts import Line, show, output_file, defaults

defaults.plot_width = 450
defaults.plot_height = 400

# build a dataset where multiple columns measure the same thing
data = dict(python=[2, 3, 7, 5, 26, 221, 44, 233, 254, 265, 266, 267, 120, 111],
            pypy=[12, 33, 47, 15, 126, 121, 144, 233, 254, 225, 226, 267, 110, 130],
            jython=[22, 43, 10, 25, 26, 101, 114, 203, 194, 215, 201, 227, 139, 160],
            test=['foo', 'bar', 'foo', 'bar', 'foo', 'bar', 'foo', 'bar', 'foo', 'bar', 'foo', 'bar', 'foo', 'bar']
            )
df = pd.DataFrame(data)

# add a column with a range of dates, as if the values were sampled then
df['date'] = pd.date_range('1/1/2015', periods=len(df.index), freq='D')

# default behavior for dataframe input is to plot each numerical column as a line
line = Line(df)

# build the line plots
line0 = Line(df, y=['python', 'pypy', 'jython'],
             title="Interpreters (y=['python', 'pypy', 'jython'])", ylabel='Duration', legend=True)

line1 = Line(df, x='date', y=['python', 'pypy', 'jython'],
             title="Interpreters (x='date', y=['python', 'pypy', 'jython'])", ylabel='Duration', legend=True)

line2 = Line(df, x='date', y=['python', 'pypy', 'jython'],
             dash=['python', 'pypy', 'jython'],
             title="Interpreters (x='date', y, dash=['python', 'pypy', 'jython'])", ylabel='Duration', legend=True)

line3 = Line(df, x='date', y=['python', 'pypy', 'jython'],
             dash=['python', 'pypy', 'jython'],
             color=['python', 'pypy', 'jython'],
             title="Interpreters (x='date', y, dash, color=['python', 'pypy', 'jython'])", ylabel='Duration', legend=True)

line4 = Line(df, x='date', y=['python', 'pypy', 'jython'],
             dash='test',
             color=['python', 'pypy', 'jython'],
             title="Interpreters (x='date', y, color=['python', 'pypy', 'jython'], dash='test') with tooltips", ylabel='Duration',
             legend=True, tooltips=[('series', '@series'), ('test', '@test')])

output_file("line_multi.html", title="line examples")

show(gridplot(line, line0, line1, line2, line3, line4, ncols=2))